In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as K

In [3]:
df=pd.read_csv('问题2-训练数据-归一化.csv')
scaler=pickle.load(P('问题2-MinMaxScaler-训练数据.csv').open('rb'))
df, scaler

(           time  10cm湿度(kg/m2)  40cm湿度(kg/m2)  100cm湿度(kg/m2)  土壤蒸发量(mm)   降水量(mm)   平均气温(℃)
 0    2012-01-01       0.328250       0.036261        0.008417   0.008720  0.002568  0.008070
 1    2012-02-01       0.269663       0.036261        0.008417   0.015132  0.002568  0.113180
 2    2012-03-01       0.237560       0.036261        0.008417   0.042062  0.024490  0.335609
 3    2012-04-01       0.186998       0.031885        0.007829   0.177738  0.006840  0.626319
 4    2012-05-01       0.364366       0.008753        0.004306   0.457297  0.056964  0.785020
 ..          ...            ...            ...             ...        ...       ...       ...
 118  2021-11-01       0.342697       0.702094        0.999609   0.030008  0.595275  0.360852
 119  2021-12-01       0.280899       0.701157        1.000000   0.016927  0.217316  0.225947
 120  2022-01-01       0.225522       0.701157        1.000000   0.004360  0.681001  0.129112
 121  2022-02-01       0.197432       0.701157        1.0000